In [ ]:
!pip install pynytimes

In [ ]:
from pynytimes import NYTAPI
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import datetime
import calendar

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
nyt = NYTAPI("DFQYRmQyXtA0xvz1ZVNLJ3rwr8b5aQhI", parse_dates=True)

In [ ]:
lists = nyt.best_sellers_lists()

In [ ]:
type(lists)

In [ ]:
lists[10]

In [ ]:
lists_array = np.array(lists)

In [ ]:
type(lists_array[0])

In [ ]:
lists_array[0]['list_name']

In [ ]:
list_encoded_names = []
for i in range(len(lists_array)):
  encoded = lists_array[i]['list_name_encoded']
  list_encoded_names.append(encoded)
list_encoded_names = np.array(list_encoded_names)

In [ ]:
list_encoded_names

In [ ]:
nyt.best_sellers_list(name="combined-print-and-e-book-fiction", date=None)

In [ ]:
test = nyt.best_sellers_list(name="combined-print-and-e-book-fiction", date=None)
print(test)

In [ ]:
test[0].keys()

In [ ]:
len(test)

In [ ]:
for i in range(len(test)-1):
  first_keys = test[i].keys()
  next_keys = test[i+1].keys()
  if first_keys == next_keys:
    print('good')
    print(i)

In [ ]:
keys = list(test[0].keys())
keys = keys[:23]
keys

In [ ]:
test[0]

In [ ]:
def list_to_df(list_stuff):
  books_data = []
  counter = 0
  for i in range(len(list_stuff)):
    values = []
    for j in enumerate(keys):
      key = j[1]
      values.append(list_stuff[counter][key])
    books_data.append(values)
    counter = counter+1
  df = pd.DataFrame(books_data, columns = keys)
  return df

In [ ]:
books_data =list_to_df(test)
books_data.head()

In [ ]:
len(books_data.columns)

In [ ]:
list_encoded_names

In [ ]:
test_codes = list_encoded_names[:4]
test_codes

In [ ]:
def categories_and_dates(df, genre, month, day):
  categories = np.empty(len(df), dtype=object)
  categories.fill(genre)
  df['categories'] = categories

  list_date = np.empty(len(df), dtype=object)
  list_date.fill([2023, month, day])
  df['list_publication_date'] = list_date

  month_names = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
  list_month = np.empty(len(df), dtype=object)
  month_name = month_names[month-1]
  list_month.fill(month_name)
  df['month'] = list_month

  return df

In [ ]:
count = 0
for i, genre in enumerate(test_codes):
  category = nyt.best_sellers_list(name=genre, date=datetime.datetime(2023, 12, 3))
  if count == 0:
    best_sellers_df = list_to_df(category)
    best_sellers_df = categories_and_dates(best_sellers_df, genre, 12, 3)
  else:
    df = list_to_df(category)
    df = categories_and_dates(df, genre, 12, 3)
    #best_sellers_df = best_sellers_df.append(df, ignore_index = True)
    best_sellers_df = pd.concat([best_sellers_df, df], ignore_index=True)
  count = count+1
  print(count)

In [ ]:
best_sellers_df.head()

In [ ]:
best_sellers_df['categories'].unique()

In [ ]:
best_sellers_df.columns

In [ ]:
months = np.arange(1, 12, 1) #Excluding December, because we already have the only data that's available for that month at this time
months

In [ ]:
def get_sundays(month):
  Cal = calendar.Calendar(firstweekday=0)
  dates = np.array(Cal.monthdays2calendar(2023, month))
  dates = dates.T[0,6]
  sundays = []
  for i, day in enumerate(dates):
    if day == 0:
      continue
    else:
      sundays.append(day)
  sundays = np.array(sundays)
  return sundays

In [ ]:
get_sundays(months[10])

In [ ]:
def month_to_df(month, genre):
  sundays = get_sundays(month)
  for i, day in enumerate(sundays):
    if i == 0:
      category = nyt.best_sellers_list(name = genre, date = datetime.datetime(2023, month, day))
      df_month = list_to_df(category)
      df_month = categories_and_dates(df_month, genre, month, day)
    else:
      category = nyt.best_sellers_list(name = genre, date = datetime.datetime(2023, month, day))
      df = list_to_df(category)
      df = categories_and_dates(df, genre, month, day)
      df_month = pd.concat([df_month, df], ignore_index=True)
  return df_month

In [ ]:
def year_to_df(genre):
  for i, month in enumerate(months):
    if i == 0:
      year_df = month_to_df(month, genre)
    else:
      month_df = month_to_df(month, genre)
      year_df = pd.concat([year_df, month_df], ignore_index=True)
  return year_df

In [ ]:
test_year = year_to_df('hardcover-fiction')
test_year

,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,...,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,categories,list_publication_date,month
0,1,1,32,0,0,038554734X,9780385547345,Doubleday,A scientist and single mother living in Califo...,0.00,...,500,https://www.amazon.com/dp/038554734X?tag=NYTBS...,,,,,,hardcover-fiction,"[2023, 1, 1]",January
1,2,2,9,0,0,0385548923,9780385548922,Doubleday,Two childhood friends follow in their fathers’...,0.00,...,500,https://www.amazon.com/dp/0385548923?tag=NYTBS...,,,,,,hardcover-fiction,"[2023, 1, 1]",January
2,3,3,15,0,0,1668002175,9781668002179,Scribner,A high school kid inherits a shed that is a po...,0.00,...,500,https://www.amazon.com/dp/1668002175?tag=NYTBS...,,,,,,hardcover-fiction,"[2023, 1, 1]",January
3,4,5,9,0,0,0063251922,9780063251922,Harper,A reimagining of Charles Dickens’s “David Copp...,0.00,...,500,https://www.amazon.com/dp/0063251922?tag=NYTBS...,,,,,,hardcover-fiction,"[2023, 1, 1]",January
4,5,6,13,0,0,059344955X,9780593449554,Random House,Musicians from different backgrounds are attra...,0.00,...,500,https://www.amazon.com/dp/059344955X?tag=NYTBS...,,,,,,hardcover-fiction,"[2023, 1, 1]",January
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,11,0,1,0,0,059354417X,9780593544174,Putnam,The 27th book in the Dirk Pitt series. A murde...,0.00,...,500,https://www.amazon.com/dp/059354417X?tag=NYTBS...,,,,,,hardcover-fiction,"[2023, 11, 26]",November
716,12,12,28,0,0,0802162177,9780802162175,Grove,Three generations of a family living on South ...,0.00,...,500,https://www.amazon.com/dp/0802162177?tag=NYTBS...,,,,,,hardcover-fiction,"[2023, 11, 26]",November
717,13,15,28,0,0,0063204150,9780063204157,Ecco,A widow working the night shift at the Sowell ...,0.00,...,500,https://www.amazon.com/dp/0063204150?tag=NYTBS...,,,,,,hardcover-fiction,"[2023, 11, 26]",November
718,14,0,1,0,0,1668050412,9781668050415,Gallery,A collection of poems exploring how love relat...,0.00,...,500,https://www.amazon.com/dp/1668050412?tag=NYTBS...,,,,,,hardcover-fiction,"[2023, 11, 26]",November


In [ ]:
for i, genre in enumerate(test_codes):
  count = 0
  year_df = year_to_df(genre)
  year_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Indep. Study 2023 -final files/'+str(genre)+'_2023.txt')
  print(count)
  print(genre)
  count = count+1

0
combined-print-and-e-book-fiction
0
combined-print-and-e-book-nonfiction
0
hardcover-fiction
0
hardcover-nonfiction


In [ ]:
other_names = list_encoded_names[4:]
len(other_names)

55

In [ ]:
count = 0
for i, genre in enumerate(other_names):
  try:
    year_df = year_to_df(genre)
    year_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Indep. Study 2023 -final files/'+str(genre)+'_2023.txt')
  except:
    continue
  print(count)
  print(genre)
  count = count+1